# US_Census_Data

In [1]:
import sys
import os
import pandas as pd
import networkx as nx
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import numpy as np
import re 
import plotly.express as px

In [2]:
# Setting up Base Folder
BASE_FOLDER = '/Users/alishakhan/Desktop/Research/POIs'
sys.path.append(BASE_FOLDER)

In [3]:
poi_name = 'School'

# Path to the folder containing CSV files
folder_path = BASE_FOLDER+'/'+poi_name

# List all files in the folder
files = sorted(os.listdir(folder_path))

# Initialize an empty list to store dataframes
list_dfs = []

# Loop through each file in the folder
for file in files:
    if file.endswith('.csv'):
        # Read the CSV file into a dataframe
        df = pd.read_csv(os.path.join(folder_path, file))
        # Append the dataframe to the list
        list_dfs.append(df)

# Get dates
dates = []
for df in list_dfs:
  dates.append(df.iloc[0,0])

# Fix Distance_Covered for POI_TRACTCE == Home_TRACTCE
for df in list_dfs:
  df.loc[df[poi_name + '_TRACTCE'] == df['Home_TRACTCE'], 'Distance_Covered (km)'] = .02

# Create list of all tracts
# Combine all weeks into a single dataframe
df_combined = pd.concat(list_dfs)

# Collect unique tract numbers
tracts = np.unique(
    df_combined[['Home_TRACTCE', poi_name + '_TRACTCE']].values)
num_tracts = np.shape(tracts)[0]

# Collect unique distances
distances = np.unique(
    df_combined[['Distance_Covered (km)']].values)

print('Number of Unique Tracts: ', num_tracts)
# print('\nDates: \n', dates)
pd.set_option('display.width', 1000)
# print('\n', df_combined.head())

Number of Unique Tracts:  485


In [4]:
CENSUS_path = '/Users/alishakhan/Desktop/Research/Census Files/Harris'
tracts_set = set(map(str, tracts))
pd.set_option('display.max_rows', None)

## Functions

In [40]:
def create_df(file_path):
    df = pd.read_csv(file_path)
    return df

def transposed_df(df):
    t_df = df.T
    return t_df

def extract_index(df):
    new_index = []
    for i in df.index:
        match = re.findall(r'\d+\.?\d*', i)
        if match:
            new_index.append(match[0])
        else:
            new_index.append('')
    df.index = new_index
    return df

Bottom_Tracts = [
    555701, 254100, 532900, 431600, 342400,
    332800, 421900, 532400, 532700, 430300,
    550800, 343200, 251600, 221900, 412600,
    251501, 430900, 251401, 553403, 430600,
    430700, 343000, 250701, 450700, 412800]

Top_Tracts = [
    430500, 552200, 554902, 251300, 420100,
    521900, 340400, 554001, 522401, 430800,
    251402, 324000, 323802, 250500, 342001,
    430400, 342700, 454600, 342300, 252700,
    542302, 551800, 322200, 222200, 411400]

def extract_index_top(df):
    new_index = []
    for i in df.index:
        match = re.findall(r'\d+\.?\d*', i)
        if match:
            new_index.append(match[0])
        else:
            new_index.append('')
    df.index = new_index
    return df

def tract_matching(top_25_tracts):
    compare =  [430500, 552200, 554902, 251300, 420100,
                521900, 340400, 554001, 522401, 430800,
                251402, 324000, 323802, 250500, 342001,
                430400, 342700, 454600, 342300, 252700,
                542302, 551800, 322200, 222200, 411400]
    matched_tracts = set(compare).intersection(top_25_tracts)
    if matched_tracts:
        print("Matched tracts")
        for tract in matched_tracts:
            print(tract)
    else: 
        print("No matching tracts")
    return matched_tracts 

def tract_matching2(top_25_tracts):
    compare =  [
    555701, 254100, 532900, 431600, 342400,
    332800, 421900, 532400, 532700, 430300,
    550800, 343200, 251600, 221900, 412600,
    251501, 430900, 251401, 553403, 430600,
    430700, 343000, 250701, 450700, 412800]
    
    matched_tracts = set(compare).intersection(top_25_tracts)
    if matched_tracts:
        print("Matched tracts")
        for tract in matched_tracts:
            print(tract)
    else: 
        print("No matching tracts")
    return matched_tracts 

def get_sparse_matrix(df, tracts): #row index = home, col index = target
    num_tracts = len(tracts)
    directed_matrix = np.zeros((num_tracts, num_tracts),dtype=float)
    for index, row in df.iterrows():
        home_index = np.where(tracts == row['Home_TRACTCE'])[0][0]
        school_index = np.where(tracts == row['School_TRACTCE'])[0][0]
        directed_matrix[home_index, school_index] = row['Visitor_Count']
    sparse_matrix = csr_matrix(directed_matrix)
    return sparse_matrix

def get_weighted_adjacency_matrices(list_dfs, tracts):
    mats = []
    for df in list_dfs:
        mats.append(get_sparse_matrix(df, tracts))
    return mats

def get_node_degrees_in(list_mats):
    degrees = []
    for mat in list_mats:
        sums = np.array(mat.sum(axis=0)).flatten()
        degrees.append(sums)
    degrees = np.array(degrees)
    return degrees

def get_node_degrees_out(list_mats):
    degrees = []
    for mat in list_mats:
        sums = np.array(mat.sum(axis=1)).flatten()
        degrees.append(sums)
    degrees = np.array(degrees)
    return degrees

## Out-Degree Change for Storm Week (Week 8)

In [71]:
schools_mats = get_weighted_adjacency_matrices(list_dfs, tracts)
# degrees_in = get_node_degrees_in(schools_mats)
degrees_out = get_node_degrees_out(schools_mats)
weekly_means_out = np.mean(degrees_out, axis=0)

np.set_printoptions(threshold=np.inf)
tract_change = []

for week in degrees_out:
    change = week - weekly_means_out
    tract_change.append(change)

e_df = pd.DataFrame({
    'Change': tract_change[7]
},index = tracts)
mats = get_weighted_adjacency_matrices(list_dfs, tracts)
base_degrees = get_node_degrees_out(mats[1:5])
base_mean = np.mean(base_degrees, axis = 0)

e_df['Base Degrees Mean'] = -base_mean

e_df = e_df.sort_values(by = 'Base Degrees Mean')

fig = px.scatter(e_df, x = e_df.index.astype(str), y = 'Change', labels={'x': 'Tract Index', 'Change': 'Change from Baseline (Weeks 1-5)'})
fig.update_layout(title='Out-Degree Change for Storm Week (Week 8)')
fig.show()

percent_e = (e_df['Change'] - e_df['Base Degrees Mean']) / e_df['Base Degrees Mean']
e_df['Percent Change from Baseline'] = percent_e

fig2 = px.scatter(e_df, x = e_df.index.astype(str), y = 'Percent Change from Baseline' , labels={'x': 'Tract Index', 'Percent Change from Baseline': 'Percent Change from Baseline (Weeks 1-5)'})
fig2.update_layout(title='Percent Out-Degree Change for Storm Week (Week 8)')
fig2.show()

In [75]:
# e_df

## Harris County

### Race

In [11]:
file = 'race_harris.csv'
file_path = CENSUS_path + '/' + file
df_h = create_df(file_path)
t_df = transposed_df(df_h)
t_df = extract_index(t_df)

t_df.index = t_df.index.map(lambda x: ''.join(x.split('.')) if '.' in x else x + '00')
t_df = t_df[t_df.index.isin(tracts_set)]
t_df.index = t_df.index.astype(str)
t_df[1] = t_df[1].str.replace(',', '')
t_df[1] = t_df[1].fillna('0')

In [178]:
t_df[1] = pd.to_numeric(t_df[1])

sorted_df = t_df.sort_values(by = 1)

top_25_white = sorted_df[1].tail(25)
top_25_white = top_25_white.index
top_25_white = top_25_white.astype(int)
tract_matching(top_25_white)
# t_df[1].tail(25)

Matched tracts
342001
251300
342300
554902


{251300, 342001, 342300, 554902}

In [13]:
t_df[2] = t_df[2].str.replace(',', '')
t_df[2] = pd.to_numeric(t_df[2])
sorted_df = t_df.sort_values(by = 2)
top_25_AA = sorted_df[2].tail(25)
top_25_AA = top_25_AA.index
top_25_AA = top_25_AA.astype(int)
tract_matching(top_25_AA)

No matching tracts


set()

In [14]:
t_df[3] = t_df[3].str.replace(',', '')
t_df[3] = pd.to_numeric(t_df[3])
sorted_df = t_df.sort_values(by = 3)
top_25_AI = sorted_df[3].tail(25)
top_25_AI = top_25_AI.index
top_25_AI = top_25_AI.astype(int)
tract_matching(top_25_AI)

No matching tracts


set()

In [15]:
t_df[4] = t_df[4].str.replace(',', '')
t_df[4] = pd.to_numeric(t_df[4])
sorted_df = t_df.sort_values(by = 4)
top_25_asian = sorted_df[4].tail(25)
top_25_asian = top_25_asian.index
top_25_asian = top_25_asian.astype(int)
tract_matching(top_25_asian)

Matched tracts
552200
430800


{430800, 552200}

In [16]:
t_df[5] = t_df[5].str.replace(',', '')
t_df[5] = pd.to_numeric(t_df[5])
sorted_df = t_df.sort_values(by = 5)
top_25_hawaiian = sorted_df[5].tail(25)
tract_matching(top_25_hawaiian)

No matching tracts


set()

In [17]:
#Missing tracts data for white pop
#980000     NaN
#980100     NaN
#324200     NaN
#340201     NaN

### Income

In [49]:
file = 'income_harris.csv'
file_path = CENSUS_path + '/' + file
df_income = create_df(file_path)
t_income = transposed_df(df_income)

#Filter the DataFrame to get the Household Median Income for all Census Tracts in Harris County for the full year of 2021
f_income = pd.DataFrame()
for index, row in t_income.iterrows():
    if "Median income (dollars)!!Estimate" in index:
        f_income = pd.concat([f_income, pd.DataFrame([row], index=[index])])
        
f_income = extract_index(f_income)
f_income.index = f_income.index.map(lambda x: ''.join(x.split('.')) if '.' in x else x + '00')
f_income = f_income[f_income.index.isin(tracts_set)]
f_income[1] = f_income[1].str.replace(',', '')
f_income[1] = f_income[1].str.replace('-','72353')
f_income[1] = f_income[1].str.replace('250000+','250000')

f_income[1] = pd.to_numeric(f_income[1])
sorted_df = f_income.sort_values(by = 1)

In [50]:
Top_Tracts =["430500", "552200", "251300", "554902", "522401", 
             "340400", "554001", "251402", "521900", "430800",
             "323802", "250500", "324000", "420100", "454600", 
             "430400", "342700", "252700", "542302", "342001",
             "342300", "322200", "222200", "551800", "233702"]

Bottom_Tracts = ["342400", "431600", "254100", "534001", 
                 "532900", "421900", "332800", "550800", 
                 "430300", "532700", "343200", "532400", 
                 "251501", "251600", "412600", "221900", 
                 "430900", "251401", "553403", "430600",
                 "430700", "343000", "250701", "450700", "412800"]

f_income_top = f_income[f_income.index.isin(Top_Tracts)]
f_income_bottom = f_income[f_income.index.isin(Bottom_Tracts)]

sorted_df_t = f_income_top.sort_values(by = 1)
sorted_df_b = f_income_bottom.sort_values(by = 1)

top_25_income = sorted_df_t[1]
bottom_25_income = sorted_df_b[1]

fig = px.scatter(top_25_income, x=top_25_income.index, y=1, title='Income by Tract (Top)', labels={ '1' : 'Median Income' , 'index' : 'Tracts'})
fig.update_yaxes(range=[10000, 260000])
fig.show()

fig2 = px.scatter(bottom_25_income, x=bottom_25_income.index, y=1, title='Income by Tract (Bottom)', labels={ '1' : 'Median Income' , 'index' : 'Tracts'})
fig2.update_yaxes(range=[10000, 260000])
fig2.show()

In [171]:
e_df = e_df.sort_index()
e_df['Income'] = f_income[1].values
e_df = e_df.sort_values(by = 'Base Degrees Mean')

fig3 = px.scatter(e_df, x = 'Income', y = 'Change', labels={'x': 'Income', 'Change': 'Change'})
fig3.update_layout(title='Out-Degree Change vs Income for Storm Week (Week 8)')
fig3.show()

fig6 = px.scatter(e_df, x = 'Income', y = 'Percent Change from Baseline', labels={'x': 'Income', 'Percent Change from Baseline': 'Percent Change from Baseline'})
fig6.update_layout(title='Percent Out-Degree Change vs Income for Storm Week (Week 8)')
fig6.show()

fig4 = px.scatter(e_df, x = 'Change', y = 'Income', labels={'x': 'Income', 'Change': 'Change'})
fig4.update_layout(title='Out-Degree Change vs Income for Storm Week (Week 8)')
fig4.show()

fig5 = px.scatter(e_df, x = 'Percent Change from Baseline', y = 'Income', labels={'x': 'Percent Change from Baseline', 'Change': 'Change'})
fig5.update_layout(title='Percent Out-Degree Change vs Income for Storm Week (Week 8)')
fig5.show()

### Unemployment Rate

In [178]:
#Making the original file into a data frame and transpose it
file = 'unemp.rate.csv'
file_path = CENSUS_path + '/' + file
df_UNemp = create_df(file_path)
t_UNemp = transposed_df(df_UNemp)

#Filter the DataFrame to get the Unemployment Rate for all Census Tracts in Harris County for the full year of 2021
f_UNemp = pd.DataFrame()
for index, row in t_UNemp.iterrows():
    if "Total population!!Percent" in index:
        f_UNemp = pd.concat([f_UNemp, pd.DataFrame([row], index=[index])])
        
#Get the Data for only those Tracts that we have from the safegraph Data and fill in the missing tracts with zeros
f_UNemp = extract_index(f_UNemp)
f_UNemp.index = f_UNemp.index.map(lambda x: ''.join(x.split('.')) if '.' in x else x + '00')
f_UNemp = f_UNemp[f_UNemp.index.isin(tracts_set)] #4 tracts are missing from this data {'340201', '454400', '980000', '980100'}
all_tracts = set(tracts_set)

# Identify the missing tracts
missing_tracts = all_tracts - set(f_UNemp.index)
missing_data = {
    'Tract': list(missing_tracts),
    'UNemp': [0, 0, 0, 0]  # Replace these values with actual data if available
}

# Concat the missing tracts (their values are manually set to 0) 
missing_df = pd.DataFrame(missing_data).set_index('Tract')
f_UNemp = pd.concat([f_UNemp, missing_df])

# print(np.shape(f_UNemp))
# print(f_UNemp[9])

In [179]:
f_UNemp[9] = f_UNemp[9].str.replace('%','')


In [180]:
#Plot the top tracts and bottom tracts unemp rate
Top_Tracts =["430500", "552200", "251300", "554902", "522401", 
             "340400", "554001", "251402", "521900", "430800",
             "323802", "250500", "324000", "420100", "454600", 
             "430400", "342700", "252700", "542302", "342001",
             "342300", "322200", "222200", "551800", "233702"]

Bottom_Tracts = ["342400", "431600", "254100", "534001", "532900",
                 "421900", "332800", "550800", "430300", "532700",
                 "343200", "532400", "251501", "251600", "412600", 
                 "221900", "430900", "251401", "553403", "430600",
                 "430700", "343000", "250701", "450700", "412800"]

#Extract the top and bottom tracts 
f_UNemp_top = f_UNemp[f_UNemp.index.isin(Top_Tracts)]
f_UNemp_bottom = f_UNemp[f_UNemp.index.isin(Bottom_Tracts)]

#Replace the "%"
f_UNemp_top.loc[:, 9] = f_UNemp_top.loc[:, 9].str.replace('%', '')
f_UNemp_bottom.loc[:, 9] = f_UNemp_bottom.loc[:, 9].str.replace('%', '')


f_UNemp_top.loc[:, 9] = pd.to_numeric(f_UNemp_top.loc[:, 9])
f_UNemp_bottom.loc[:, 9] = pd.to_numeric(f_UNemp_bottom.loc[:, 9])

# Sort the top and Bottom Tracts by values
sorted_df_t = f_UNemp_top.sort_values(by = 9)
sorted_df_b = f_UNemp_bottom.sort_values(by = 9)

#Extract only the 9th Column
top_25_emp = sorted_df_t[9]
bottom_25_emp = sorted_df_b[9]

# Plot the Values
fig = px.scatter(top_25_emp, x=top_25_emp.index, y=9, title='Unemployment Rate by Tract (Top 25)', labels={ '9' : 'Rate (%)' , 'index' : 'Tracts'})
fig.update_yaxes(range=[-10, 25])
fig.show()

fig2 = px.scatter(bottom_25_emp, x=bottom_25_emp.index, y=9, title='Unemployment Rate by Tract (Bottom 25)', labels={ '9' : 'Rate (%)' , 'index' : 'Tracts'})
fig2.update_yaxes(range=[-10, 25])
fig2.show()

In [183]:
f_UNemp[9] = pd.to_numeric(f_UNemp[9])

e_df = e_df.sort_index()
e_df['Unemployment Rate'] = f_UNemp[9].values
e_df = e_df.sort_values(by = 'Base Degrees Mean')

fig3 = px.scatter(e_df, x = 'Unemployment Rate', y = 'Change', labels={'x': 'Unemployment Rate (%)', 'Change': 'Change'})
fig3.update_layout(title='Out-Degree Change vs Unemployment Rate for Storm Week (Week 8)')
fig3.show()

fig6 = px.scatter(e_df, x = 'Unemployment Rate', y = 'Percent Change from Baseline', labels={'x': 'Unemployment Rate (%)', 'Percent Change from Baseline': 'Percent Change from Baseline'})
fig6.update_layout(title='Percent Out-Degree Change vs Unemployment Rate for Storm Week (Week 8)')
fig6.show()

fig4 = px.scatter(e_df, x = 'Change', y = 'Unemployment Rate', labels={'x': 'Unemployment Rate (%)', 'Change': 'Change'})
fig4.update_layout(title='Out-Degree Change vs Unemployment Rate for Storm Week (Week 8)')
fig4.show()

fig5 = px.scatter(e_df, x = 'Percent Change from Baseline', y = 'Unemployment Rate', labels={'x': 'Percent Change from Baseline', 'Unemployment Rate': 'Unemployment Rate (%)'})
fig5.update_layout(title='Percent Out-Degree Change vs Unemployment Rate for Storm Week (Week 8)')
fig5.show()

### Poverty Percentage

In [29]:
file = 'poverty.csv'
file_path = CENSUS_path + '/' + file
df_pov = create_df(file_path)
t_pov = transposed_df(df_pov)
t_pov

#Filter the DataFrame to get the Unemployment Rate for all Census Tracts in Harris County for the full year of 2021
f_pov = pd.DataFrame()
for index, row in t_pov.iterrows():
    if "!!Percent below poverty level!!Estimate" in index:
        f_pov = pd.concat([f_pov, pd.DataFrame([row], index=[index])])

f_pov = extract_index(f_pov)
f_pov.index = f_pov.index.map(lambda x: ''.join(x.split('.')) if '.' in x else x + '00')
f_pov = f_pov[f_pov.index.isin(tracts_set)]
f_pov[0] = f_pov[0].str.replace('-','')
f_pov[0] = f_pov[0].str.replace('%','')
f_pov[0] = pd.to_numeric(f_pov[0])
sorted_df = f_pov.sort_values(by = 0)
top_25_pov = sorted_df[0].tail(25)
top_25_pov = top_25_pov.index
top_25_pov = top_25_pov.astype(int)
tract_matching(top_25_pov)

No matching tracts


set()

### Education

In [30]:
file = 'education.csv'
file_path = CENSUS_path + '/' + file
df_edu = create_df(file_path)
t_edu = transposed_df(df_edu)
t_edu[14]

# Filter the DataFrame to obtain the percentage of the population aged 25 years or older who have a high school diploma or higher degree 
# for all Census Tracts in Harris County for the entire year of 2021.

f_edu = pd.DataFrame()
for index, row in t_edu.iterrows():
    if "Texas!!Percent!!Estimate" in index:
        f_edu = pd.concat([f_edu, pd.DataFrame([row], index=[index])])

f_edu = extract_index(f_edu)
f_edu.index = f_edu.index.map(lambda x: ''.join(x.split('.')) if '.' in x else x + '00')
f_edu = f_edu[f_edu.index.isin(tracts_set)]
f_edu[14] = f_edu[14].str.replace('-','')
f_edu[14] = f_edu[14].str.replace('%','')
f_edu[14] = pd.to_numeric(f_edu[14])
sorted_df = f_edu.sort_values(by = 14)
top_25_edu = sorted_df[14].tail(25)
top_25_edu = top_25_edu.index
top_25_edu = top_25_edu.astype(int)
tract_matching(top_25_edu)

Matched tracts
430400
430500
430800


{430400, 430500, 430800}

### Older Population

In [31]:
#Download file from census website (https://data.census.gov/) and create a file path
census_folder_path = '/Users/alishakhan/Desktop/Research/Census Files/Harris'
file_name = 'olderpop_harris.csv'
file_path = os.path.join(census_folder_path, file_name)

#Create a Dataframe for the file
df_o = pd.read_csv(file_path)

#Transpose the file (in order to get rows of census tracts instead of columns)
transposed_df_o = df_o.T

#Filter the DataFrame to get the Population 65 Years Old and Over for all Census Tracts in Harris County for the full year of 2021
filtered_df_o = pd.DataFrame()
for index, row in transposed_df_o.iterrows():
    if "Total!!Estimate" in index:
        filtered_df_o = pd.concat([filtered_df_o, pd.DataFrame([row], index=[index])])

# Rename column '24' to '65 Years and Over'
filtered_df_o = filtered_df_o.rename(columns={31: '65 Yrs and Over'})

#Extract from the Dataframes index the Census-Tract Number and reassign it again to the index
filtered_df_o.index = [re.findall(r'\d+\.?\d*', i)[0] for i in filtered_df_o.index]

#Adjust the Census-Tract Numbers: remove any decimal points if present; if no decimal points exist, append '00'
filtered_df_o.index = filtered_df_o.index.map(lambda x: ''.join(x.split('.')) if '.' in x else x + '00')

#Convert Safegraph tracts into a set
tracts_set = set(map(str, tracts))

#Using the Safegraph tracts set, extract income data for only those tracts
filtered_df_o_t = filtered_df_o[filtered_df_o.index.isin(tracts_set)]

#Final Result
print('Tract   65 Yrs and Over')
filtered_df_o_t.index.name = None

# filtered_df_h_t.index.name = 'Tract'
# print((filtered_df_h_t[['Income']].head(15)))

print((filtered_df_o_t['65 Yrs and Over'].head(15)))

Tract   65 Yrs and Over
210400      600
210500      710
210600    1,068
210700      383
210800      252
210900      207
211000      398
211200      267
211400      271
211600      420
211700      457
211900      822
212300      460
212400      400
212500      623
Name: 65 Yrs and Over, dtype: object


In [32]:
#Checking Tracts are the same size
print(np.size(filtered_df_o_t['65 Yrs and Over']))
print(len(tracts_set))

485
485


In [33]:
# fig = px.bar(filtered_df_o_t, x=filtered_df_o_t.index, y='65 years and over', title='Older Population')
# fig.show()

filtered_df_o_t.loc[:,'65 Yrs and Over'] = filtered_df_o_t['65 Yrs and Over'].str.replace(',', '').astype(int)
filtered_df_o_t_s = filtered_df_o_t.sort_values(by='65 Yrs and Over')
fig = px.bar(filtered_df_o_t_s, x=filtered_df_o_t_s.index, y='65 Yrs and Over', title='Population 65 Years and Over by Tract')
fig.show()